In [1]:
import glob
import numpy as np
import pandas as pd

## Read CSVs into DataFrame

2008-2012
https://www.misoenergy.org/Library/MarketReports/Pages/ArchivedHistoricalHourlyWindData.aspx

2013-2015
https://www.misoenergy.org/RTDisplays/MKTRPT/AllReportsList.html?rptName=Historical%20Hourly%20Wind%20Data%20(csv)

In [2]:
# Get list of all LMP CSV files
path = '../data/raw/MISO - Wind Generation'
allFilePaths = glob.glob(path + "/*.csv")

for filePath in allFilePaths:
    print filePath

../data/raw/MISO - Wind Generation/20081231_hwd_HIST.csv
../data/raw/MISO - Wind Generation/20091231_hwd_HIST.csv
../data/raw/MISO - Wind Generation/20101231_hwd_HIST.csv
../data/raw/MISO - Wind Generation/20111231_hwd_HIST.csv
../data/raw/MISO - Wind Generation/20121231_hwd_HIST.csv
../data/raw/MISO - Wind Generation/20131231_hwd_HIST.csv
../data/raw/MISO - Wind Generation/20141231_hwd_hist.csv
../data/raw/MISO - Wind Generation/20151121_hwd_hist.csv


In [3]:
# Treat the 2 sets of files separately as they have different formats
filePaths1 = ['../data/raw/MISO - Wind Generation/20081231_hwd_HIST.csv',
'../data/raw/MISO - Wind Generation/20091231_hwd_HIST.csv',
'../data/raw/MISO - Wind Generation/20101231_hwd_HIST.csv',
'../data/raw/MISO - Wind Generation/20111231_hwd_HIST.csv']

filePaths2 = ['../data/raw/MISO - Wind Generation/20121231_hwd_HIST.csv',
             '../data/raw/MISO - Wind Generation/20131231_hwd_HIST.csv',
             '../data/raw/MISO - Wind Generation/20141231_hwd_HIST.csv',
             '../data/raw/MISO - Wind Generation/20151121_hwd_HIST.csv']

## Wind Generation 2008-2010

In [4]:
dfWind = pd.DataFrame()

In [5]:
for filePath in filePaths1:
    # Read CSV as time series
    df = pd.read_csv(filePath, skiprows=7, error_bad_lines=False)
    df.columns = ['date', 'windGenerationMWh']
    df = df.set_index('date')
    df.index = pd.to_datetime(df.index, format='%m/%d/%y %I:%M %p')
    df = df.sort_index()

    # Resample to average daily power generation
    df = df.resample('1D', how='mean')
    
    # Append to main DataFrame
    dfWind = pd.concat([dfWind, df])
    
dfWind.head()

Skipping line 8793: expected 2 fields, saw 10

Skipping line 8770: expected 2 fields, saw 10

Skipping line 8770: expected 2 fields, saw 10

Skipping line 8770: expected 2 fields, saw 10



,windGenerationMWh
date,
2008-01-01,1176.730000
2008-01-02,617.977917
2008-01-03,1267.323333
2008-01-04,1285.813750
2008-01-05,1127.440833


## Wind Generation 2012-2015

In [6]:
for filePath in filePaths2:
    # Read CSV as time series
    df = pd.read_csv(filePath, skiprows=7, error_bad_lines=False)
    df.columns = ['date', 'Hour Ending', 'windGenerationMWh']
    df = df.drop('Hour Ending', axis=1)
    df = df.set_index('date')
    df.index = pd.to_datetime(df.index)
    df = df.sort_index()

    # Resample to average daily power generation
    df = df.resample('1D', how='mean')
    
#     Append to main DataFrame
    dfWind = pd.concat([dfWind, df])
    
df.head()

Skipping line 8769: expected 3 fields, saw 10

Skipping line 8769: expected 3 fields, saw 10

Skipping line 7785: expected 3 fields, saw 10



,windGenerationMWh
date,
2015-01-01,6774.195833
2015-01-02,2735.098750
2015-01-03,5281.879167
2015-01-04,8827.305833
2015-01-05,4181.999167


## Write to CSV

In [7]:
dfWind.to_csv('../data/MISOWindGeneration.csv')